In [1]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et


In [3]:
#loading xml and storing it in list
xml_list = glob('./train/*.xml')
#data cleaning
xmlfiles = []
xmlfiles = list(map(lambda x:x.replace("\\","/"),xml_list))
xmlfiles

['./train/EYE DOWN_1.xml',
 './train/EYE DOWN_10.xml',
 './train/EYE DOWN_11.xml',
 './train/EYE DOWN_12.xml',
 './train/EYE DOWN_13.xml',
 './train/EYE DOWN_14.xml',
 './train/EYE DOWN_15.xml',
 './train/EYE DOWN_16.xml',
 './train/EYE DOWN_17.xml',
 './train/EYE DOWN_18.xml',
 './train/EYE DOWN_19.xml',
 './train/EYE DOWN_2.xml',
 './train/EYE DOWN_20.xml',
 './train/EYE DOWN_21.xml',
 './train/EYE DOWN_22.xml',
 './train/EYE DOWN_23.xml',
 './train/EYE DOWN_24.xml',
 './train/EYE DOWN_25.xml',
 './train/EYE DOWN_26.xml',
 './train/EYE DOWN_27.xml',
 './train/EYE DOWN_28.xml',
 './train/EYE DOWN_29.xml',
 './train/EYE DOWN_3.xml',
 './train/EYE DOWN_30.xml',
 './train/EYE DOWN_31.xml',
 './train/EYE DOWN_32.xml',
 './train/EYE DOWN_33.xml',
 './train/EYE DOWN_34.xml',
 './train/EYE DOWN_35.xml',
 './train/EYE DOWN_36.xml',
 './train/EYE DOWN_37.xml',
 './train/EYE DOWN_38.xml',
 './train/EYE DOWN_39.xml',
 './train/EYE DOWN_4.xml',
 './train/EYE DOWN_40.xml',
 './train/EYE DOWN_41.xm

In [4]:
#read xml file, extract filename, width, height, xmin, xmax, ymin, ymax
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()
    image_name = root.find('filename').text
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name,width,height,name,xmin,xmax,ymin,ymax])
    return parser

In [5]:
extract_text('./train/EYE DOWN_14.xml')

[['EYE DOWN_14.jpg', '800', '600', 'EYE DOWN', '148', '256', '325', '433'],
 ['EYE DOWN_14.jpg', '800', '600', 'EYE DOWN', '538', '635', '324', '419']]

In [6]:
parser_all = list(map(extract_text,xmlfiles))
# parser_all


In [7]:
data = reduce(lambda x,y : x+y, parser_all) 
    

In [8]:
df = pd.DataFrame(data,columns=['filename','width','height','name','xmin','xmax','ymin','ymax'])
df

,filename,width,height,name,xmin,xmax,ymin,ymax
0,EYE DOWN_1.jpg,800,600,EYE DOWN,101,279,367,494
1,EYE DOWN_1.jpg,800,600,EYE DOWN,571,725,403,521
2,EYE DOWN_10.jpg,800,600,EYE DOWN,161,256,156,270
3,EYE DOWN_10.jpg,800,600,EYE DOWN,501,595,193,294
4,EYE DOWN_11.jpg,800,600,EYE DOWN,204,284,227,308
...,...,...,...,...,...,...,...,...
713,EYE UP_89.jpg,4624,3472,EYE UP,3091,4314,1518,2772
714,EYE UP_9.jpg,800,600,EYE UP,131,256,278,369
715,EYE UP_9.jpg,800,600,EYE UP,506,640,278,379
716,EYE UP_90.jpg,4624,3472,EYE UP,1573,2764,1613,2622


In [9]:
df.shape

(718, 8)

In [10]:
df['name'].value_counts()

name
EYE DOWN     181
EYE RIGHT    180
EYE UP       180
EYE LEFT     177
Name: count, dtype: int64

In [11]:
cols = ['width', 'height', 'xmin', 'xmax', 'ymin', 'ymax']  # Exclude 'name' if it's a string
df[cols] = df[cols].astype(int)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718 entries, 0 to 717
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  718 non-null    object
 1   width     718 non-null    int64 
 2   height    718 non-null    int64 
 3   name      718 non-null    object
 4   xmin      718 non-null    int64 
 5   xmax      718 non-null    int64 
 6   ymin      718 non-null    int64 
 7   ymax      718 non-null    int64 
dtypes: int64(6), object(2)
memory usage: 45.0+ KB


In [12]:
#center x, center y
df['center_x'] = ((df['xmin']+df['xmax'])/2)/df['width']
df['center_y'] = ((df['ymin']+df['ymax'])/2)/df['height']

In [13]:
#w #h
df['w'] = (df['xmax']-df['xmin'])/df['width']
df['h'] = (df['ymax']-df['ymin'])/df['height']

In [14]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,EYE DOWN_1.jpg,800,600,EYE DOWN,101,279,367,494,0.237500,0.717500,0.22250,0.211667
1,EYE DOWN_1.jpg,800,600,EYE DOWN,571,725,403,521,0.810000,0.770000,0.19250,0.196667
2,EYE DOWN_10.jpg,800,600,EYE DOWN,161,256,156,270,0.260625,0.355000,0.11875,0.190000
3,EYE DOWN_10.jpg,800,600,EYE DOWN,501,595,193,294,0.685000,0.405833,0.11750,0.168333
4,EYE DOWN_11.jpg,800,600,EYE DOWN,204,284,227,308,0.305000,0.445833,0.10000,0.135000


In [15]:
#label encoding
df.rename(columns={'name': 'class'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718 entries, 0 to 717
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   filename  718 non-null    object 
 1   width     718 non-null    int64  
 2   height    718 non-null    int64  
 3   class     718 non-null    object 
 4   xmin      718 non-null    int64  
 5   xmax      718 non-null    int64  
 6   ymin      718 non-null    int64  
 7   ymax      718 non-null    int64  
 8   center_x  718 non-null    float64
 9   center_y  718 non-null    float64
 10  w         718 non-null    float64
 11  h         718 non-null    float64
dtypes: float64(4), int64(6), object(2)
memory usage: 67.4+ KB


In [16]:
df['class'] = df['class'].str.replace(r'_\d+', '', regex=True)
df['class'].unique()

array(['EYE DOWN', 'EYE LEFT', 'EYE RIGHT', 'EYE UP'], dtype=object)

In [17]:

required_columns = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
df_selected = df[required_columns]

# Save the selected columns to a CSV file
df_selected.to_csv('train_label1.csv', index=False)